In [1]:
import numpy as np
import pandas as pd
from OLG_model_src import Parameters2, OLG
import altair as alt


%load_ext autoreload
%autoreload 2

In [2]:
countrydata_df = pd.read_csv('countrydata_df.csv', parse_dates=['date'])[['Country', 'StringencyIndex', 'date', 'I']]

In [3]:
contagion_pi_country_dict = {'Austria' :-0.21,
'Belgium':-0.24,
'Bulgaria':-0.88,
'Croatia':-0.2,
'Cyprus':-0.06,
'Denmark':-0.21,
'Finland':-0.16,
'France':-0.39
}


scenario = {'t': {0: 20, 1: 10},
            'R0D': {0: 0, 1: 0}}

p = Parameters2(tau=14, 
                init_infected=100, 
                fi=0.25, 
                theta=0.0771, 
                countries=['Israel'], 
                scenario=scenario, 
                comparator_country='Germany', 
                contagion_pi_country=contagion_pi_country_dict.get('France'))

In [17]:
# plese note that R is moving average
olg = OLG(countrydata_df, p)

In [11]:
olg.tmp

,comparator_Stringency,corona_days
1330,14.0,1
1331,14.0,2
1332,14.0,3
1333,14.0,4
1334,14.0,5
1335,14.0,6
1336,14.0,7
1337,14.0,8
1338,14.0,9
1339,29.0,10


In [9]:
olg.rma

array([0.69999853, 2.26666512, 7.68852281, 1.48148066, 5.69377896,
       1.523809  , 0.04294435, 4.06727773, 7.99526014, 0.54901939,
       3.19013037, 3.09456245, 4.97386235, 4.73999987, 3.22732115,
       2.00611621, 2.16313725, 2.84404925, 2.72006991, 1.54867257,
       1.85034315, 2.24825662, 2.25393983, 1.14480409, 1.13181019,
       1.38384541, 1.34668966, 1.12352337, 0.96303721, 0.        ,
       2.45118883, 3.9672989 , 3.30806884, 2.00076686, 2.6280765 ,
       2.75574213, 2.19125618, 1.51158529, 1.07500802, 1.36144853,
       0.83009354, 0.29286293, 0.2154277 , 0.        , 2.45883446,
       3.99857285, 3.3239579 , 2.01291007, 2.64937021, 2.76940277])

In [ ]:
countrydata_df.query('Country=="France" & I > 100')

In [ ]:
pd.set_option('display.column_space', 25)

olg.df[37:43][['R', 'I', 'corona_days','prediction_ind']]

In [ ]:
# var_in_multi_line selects one of the 3 variables: I, E, A in the country comparison plot
olg.plot_data(p.countries, var_in_multi_line='I')

In [ ]:
# due to sequantial days witout a change  some R_contagion_score behaved iradicly, so a limit of <10 on the variable was used 
olg.plot_data_contagion_score()

In [ ]:
from statsmodels.tsa.api import SimpleExpSmoothing, Holt

In [ ]:
saledata = olg.df[olg.df['r_values'].notnull()]['r_values']
saledata[-14:]

In [ ]:
fit1 = Holt(saledata).fit(smoothing_level=0.8, smoothing_slope=0.2, optimized=False)
fcast1 = fit1.forecast(30).rename("Holt's linear trend")

fit2 = Holt(saledata[-14:], exponential=True).fit(smoothing_level=0.8, smoothing_slope=0.2, optimized=False)
fcast2 = fit2.forecast(30).rename("Exponential trend")

fit3 = Holt(saledata, exponential=True).fit(smoothing_level=0.8, smoothing_slope=0.2)
fcast3 = fit3.forecast(30).rename("Additive damped trend")


fit1.fittedvalues.plot(marker="o", color='blue')
fcast1.plot(color='blue', marker="o", legend=True)

fit2.fittedvalues.plot(marker="x", color='red')
fcast2.plot(color='red', marker="x", legend=True)

fit3.fittedvalues.plot(marker=".", color='green')
fcast3.plot(color='green', marker=".", legend=True)

# plt.show()

In [ ]:
fcast3